In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql

WITH usa_tracks_sold AS
    (
    Select il.* FROM invoice_line AS il
    INNER JOIN invoice AS i ON il.invoice_id = i.invoice_id
    INNER JOIN customer AS c ON i.customer_id = c.customer_id
    WHERE c.country='USA')

SELECT g.name AS genre,
       COUNT(uts.invoice_line_id) AS tracks_sold,
       CAST(COUNT(uts.invoice_line_id) AS Float)/(SELECT COUNT(*) FROM usa_tracks_sold) AS percentage_sold
FROM usa_tracks_sold AS uts
INNER JOIN track AS t ON uts.track_id = t.track_id
INNER JOIN genre AS g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;



Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


The result shows that Rock music is the most populate type of music. Hip Hop music rank 9th and it's the last populate music.

Based on the result, we select the following artist to add into the new deal:
- Red Tone
- Meteor and the Girls
- Slim Jim Bites

In [4]:
%%sql
SELECT e.first_name || " " || e.last_name  AS employee_name,
       COUNT(c.customer_id) AS Customer_numbers,
       SUM(i.total) AS Total_Purchase,
       SUM(i.total)/CAST(COUNT(c.customer_id) AS Float) AS Purchase_per_customer
FROM employee AS e
INNER JOIN customer AS c ON e.employee_id = c.support_rep_id
INNER JOIN invoice AS i ON c.customer_id=i.customer_id
GROUP BY 1
ORDER BY 3 DESC;

Done.


employee_name,Customer_numbers,Total_Purchase,Purchase_per_customer
Jane Peacock,212,1731.5100000000039,8.167500000000018
Margaret Park,214,1584.0000000000034,7.401869158878521
Steve Johnson,188,1393.920000000002,7.414468085106393


It shows that Jane has served the highest amount of customers and the average amount of customer spend has the highest as well. Margaret has served more customers while both Margaret and Steve have the similar level of purhcase per customers.

In [5]:
%%sql
WITH other_countries AS
    (
    SELECT
      CASE
        WHEN(
            SELECT COUNT(*)
            FROM customer
            WHERE country=c.country
        ) = 1 THEN 'Other'
        ELSE c.country
    END AS country,
    c.customer_id,
    il.*
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id)

SELECT
     country,
     customers,
     total_sales,
     average_order,
     customer_lifetime_value

FROM
    (
    SELECT
         country,
         COUNT(distinct customer_id) AS customers,
         SUM(unit_price) AS total_sales,        
         SUM(unit_price) / COUNT(distinct customer_id) AS customer_lifetime_value,
         SUM(unit_price) / COUNT(distinct invoice_id) AS average_order,
         CASE
             WHEN country = 'Other' THEN 1
             ELSE 0
         END AS sort
      FROM other_countries
      GROUP BY country
      ORDER BY sort ASC, total_sales DESC
     );
        
   

        

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [6]:
%%sql
WITH invoice_first_track AS
    (
    SELECT
       il.invoice_id AS invoice_id,
       MIN(il.track_id) AS first_track_id
    FROM invoice_line AS il
    GROUP By 1
    )
    
SELECT
    album_purchase,
    COUNT(invoice_id) AS number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / ( 
                                        SELECT COUNT(*) FROM invoice
                                        ) AS percent
FROM
    (SELECT
         ifs.*,
         CASE
         WHEN
             (Select t.track_id FROM track AS t
              WHERE t.album_id = (
                                  SELECT t2.album_id FROM track AS t2
                                  WHERE t2.track_id = ifs.first_track_id
                                 )
             EXCEPT
             
             SELECT il2.track_id FROM invoice_line AS il2
             WHERE il2.invoice_id = ifs.invoice_id) IS NULL
         AND
             (SELECT il2.track_id FROM invoice_line AS il2
              WHERE il2.invoice_id = ifs.invoice_id
             EXCEPT
             
             SELECT t.track_id FROM track AS t
             WHERE t.album_id = (
                                  SELECT t2.album_id FROM track AS t2
                                  WHERE t2.track_id = ifs.first_track_id
                                 )) IS NULL
        THEN 'yes'
        ELSE 'no'
     END AS 'album_purchase'
    FROM invoice_first_track AS ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


It shows that around 18.6% of customers purchase full albums while over 80% of customers don't purchase albums. I receommend Chinook should focus on promotion of purchasing indicivual tracks to increase potential revenue.